In [1]:
import struct

In [88]:
# данные будут представлены как некоторые данные об ученике: имя, номер школы, средний, балл аттестата
DATA = [
    {
        "name": "Alexander",
        "school": 74,
        "average_score": 4.78
    },
    {
        "name": "Maxim",
        "school": 45,
        "average_score": 3.67
    },
    {
        "name": "Nikolai",
        "school": 65,
        "average_score": 4.92
    }
]

In [3]:
FILENAME = "students.bin"

In [145]:
class BinaryInterface:
    def __init__(self, filename: str):
        if filename.split('.')[-1] != 'bin':
            raise AssertionError('File must be a binary!')
        self.filename = filename
        self.bin_pattern = '>16s I f'
        
    @staticmethod
    def __get_fixed_len_string(string: str) -> str:
        string = string.strip(' ')
        if len(string) > 16:
            string = string[:16]
        elif len(string) < 16:
            space_count = 16 - len(string)
            string += ' ' * space_count
        
        return string
        
    def write_data(self, data: list[dict]):
        lines = []
        for student in data:
            if len(student['name'].strip()) > 16:
                student['name'] = student['name'][:16]
            elif len(student['name'].strip()) < 16:
                student['name'] = self.__get_fixed_len_string(student['name'])

            lines.append(struct.pack(self.bin_pattern, student['name'].encode('utf-8'), student['school'], student['average_score']))

        try:
            with open(self.filename, 'wb') as file:
                file.writelines(lines)
        except Exception as e:
            print(f"Some error occured: {e}")
            return
            
    def read_data(self) -> list[dict]:
        lines = []
        result_data = []
        try:
            with open(self.filename, 'rb') as file:
                while True:
                    data = file.readline(struct.calcsize(self.bin_pattern))
                    if not data:
                        break
                    lines.append(struct.unpack(self.bin_pattern, data))
                    
            for line in lines:
                student = {"name": line[0].decode('utf-8').strip(' '), "school": int(line[1]),
                           "average_score": float(round(line[2], 2))}
                result_data.append(student)
        except Exception as e:
            print(f"Some error occured: {e}")
            return [{}]
            
        return result_data

In [146]:
binary = BinaryInterface(FILENAME)
binary.write_data(DATA)
binary.read_data()

[{'name': 'Alexander', 'school': 74, 'average_score': 4.78},
 {'name': 'Maxim', 'school': 45, 'average_score': 3.67},
 {'name': 'Nikolai', 'school': 65, 'average_score': 4.92}]